In [3]:
import pandas as pd
import numpy as np
import glob

data_root = "../datasets/surveys/2016/"
normal_data = "*.csv"
FILENAMES = glob.glob(data_root + normal_data)

In [4]:
"""
Get unique column names.
I only ran this once and made the new_names dictionary below from the result
"""

unique_cols = set()

for file in FILENAMES:
    df = pd.read_csv(file)
    unique_cols.update(df.columns)
    
unique_cols

{'#',
 'How likely is it that you would recommend the Make School Summer Academy to a friend?',
 'How well are the tutorials paced?',
 'How well is the schedule paced?',
 'How would you rate your overall satisfaction with the Summer Academy this week?',
 'Network ID',
 'Start Date (UTC)',
 'Submit Date (UTC)',
 'Timestamp',
 'Unnamed: 0',
 'What track are you in?',
 'location',
 'track'}

In [5]:
"""
dictionary of possible names and what we want to rename them to
I only rename the ones I want to keep
"""

new_names = {
    'How well are the tutorials paced?': 'pacing',
    'How likely is it that you would recommend the Make School Summer Academy to a friend?': 'satisfaction',
    'How well are the tutorials paced?': 'pacing',
    'How well is the schedule paced?':'pacing',
    'How would you rate your overall satisfaction with the Summer Academy this week?': 'satisfaction',
    'What track are you in?':'track',
    'Timestamp':'timestamp',
    'Unnamed: 0': 'timestamp',
    'Week':'week',
    'Location': 'location'}

In [6]:
import re

# TODO: DRYify
def extract_week(filename):
    """takes a list of filenames and returns a list of weeks
        example file:
            '../datasets/surveys/2016/Anon_Week_6_Feedback_-_SG.csv'
            should return: '6'
    """
    try:
        result = re.search(r'Week_+(\d)', filename).group(1)
        return result
    except AttributeError:
        return np.NaN

def extract_location(filename):
    """takes a list of filenames and returns a list of locations
        example file:
            '../datasets/surveys/2016/Anon_Week_6_Feedback_-_SG.csv'
            should return: 'SG'
    """
    try:
        result = re.search(r'([A-Za-z])+\.csv', filename).group(0)
        return result[:-4]
    except AttributeError:
        return np.NaN
        

In [7]:
def normalize_columns(df, cols_list):
    """takes a dataframe: drops columns it doesn't need and adds new NaN filled ones.
    Arguments:
        1. df: the dataframe to normalize
        2. cols_list: list of columns the df should have by the end
    Steps:
        1. removes unnecessary columns:
        2. adds necessary columns filled with NaN
    """
    
    # remove unnecessary columns
    for col in df.columns:
        # drop the column if it wasn't renamed
        if col not in new_names.values():
            df.drop(columns=col, inplace=True)
            
    # add nonexistent columns 
    for col in cols_list:
        if col not in df.columns:
            df[col] = np.NaN
    return df

In [8]:
def SA_normalize(file, new_names):
    """takes: a filename from the summer academy dataset,
    returns a normalized dataframe"""
    # create a dataframe from the csv
    df = pd.read_csv(file)
    
    # get week and location from filename
    week = extract_week(file)
    location = extract_location(file)
    df['location'] = location
    df['week'] = week

    # rename cols
    df.rename(columns=new_names, inplace=True)

    # normalize it
    cols = ['week', 'location', 'pacing', 'track', 'timestamp']
    df = normalize_columns(df, cols)
    
    return df

In [9]:
"""goes through each file, creates a normalized dataframe, and then concats this list into one big df"""

dfs = list()
df = None # temporary
for file in FILENAMES:
    # create a dataframe from the csv
    df = SA_normalize(file, new_names)
    dfs.append(df)
    
cleaned = pd.concat(dfs, sort=False)

# normalize the timestamp to a pandas Timestamp object
cleaned['timestamp'] = pd.to_datetime(cleaned.timestamp)

df = cleaned[cleaned["track"] != "Average:"]
df

,timestamp,track,satisfaction,pacing,location,week
0,2016-07-22 12:52:43,Apps,4,3,SV,5
1,2016-07-22 12:53:47,Apps,5,3,SV,5
2,2016-07-23 14:14:16,Apps,3,3,SV,5
3,2016-07-22 14:37:17,Apps,2,3,SV,5
4,2016-07-22 12:49:36,Games,4,3,SV,5
5,2016-07-22 12:50:23,Games,4,3,SV,5
6,2016-07-22 12:53:36,Apps,5,3,SV,5
7,2016-07-22 13:11:42,Apps,3,3,SV,5
8,2016-07-22 14:03:30,Apps,5,3,SV,5
9,2016-07-22 14:25:23,Apps,4,4,SV,5


In [10]:
df.to_csv("2016.csv")